In [82]:
from test_data import gapped_sequences, sequences
from benchmark import Benchmark
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC, generic_dna, _verify_alphabet
biopy_benchmark = Benchmark("biopython")
 


biopy_seqs = [Seq(seq, generic_dna) for ident, seq in sequences]
gapped_biopy_seqs = [Seq(seq, generic_dna) for ident, seq in gapped_sequences]

In [83]:
@biopy_benchmark
def object_creation():
    for ident, seq in sequences:
        Seq(seq, generic_dna)

10000 loops, best of 3: 58.6 µs per loop


In [84]:
@biopy_benchmark
def object_creation_validate():
    for ident, seq in sequences:
        _verify_alphabet(Seq(seq, IUPAC.IUPACAmbiguousDNA))

10 loops, best of 3: 48.9 ms per loop


In [85]:
# @biopy_benchmark
# def read_fasta_file():
#     pass

In [86]:
@biopy_benchmark
def reverse_complement():
    for seq in biopy_seqs:
        seq.reverse_complement()

1000 loops, best of 3: 683 µs per loop


In [87]:
# @biopy_benchmark
# def search_for_motif():
#     pass

In [88]:
# @biopy_benchmark
# def translate():
#     pass

In [89]:
# @biopy_benchmark
# def reverse_translate():
#     pass

In [90]:
# @biopy_benchmark
# def filter_fasta_to_no_gaps():
#     pass

In [91]:
@biopy_benchmark
def degap_all():
    for seq in gapped_biopy_seqs:
        seq.ungap('-')

100 loops, best of 3: 11 ms per loop


In [92]:
# @biopy_benchmark
# def kmer_count_5():
#     pass

In [93]:
# @biopy_benchmark
# def kmer_count_25():
#     pass

In [94]:
# @biopy_benchmark
# def validate_chars():
#     pass

In [95]:
# @biopy_benchmark
# def filter_invalid_seqs():
#     pass

In [96]:
# @biopy_benchmark
# def rc_find_motif():
#     pass

In [97]:
# @biopy_benchmark
# def expand_degenerates():
#     pass

In [98]:
# @biopy_benchmark
# def gc_content():
#     pass

In [99]:
# @biopy_benchmark
# def find_motif_in_gapped():
#     pass

In [100]:
biopy_benchmark.record("./biopy.csv")